In [66]:
import re
import requests
import pandas as pd
from ast import literal_eval
from bs4 import BeautifulSoup
from datetime import datetime, timedelta


header = {
  "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
  "X-Requested-With": "XMLHttpRequest"
}

In [68]:
"""
Two steps: 
1. get gas prices from list in html with soup
2. get station info from requests
3. Combine and $$$
"""
now = datetime.now()
locations = ['Oakville', 'Burlington', 'Hamilton', 'Toronto', 'Milton', 'Mississauga']

url = 'https://www.gasbuddy.com/GasPrices/Ontario/Oakville'
url_soup = requests.get(url)

# Prices list
soup = BeautifulSoup(url_soup.content, "html.parser")
# Find prices
prices = soup.text.split('p.a = ')[1].split(';\r')[0].replace('[', '').replace(']', '').split(',"')
# Clean and remove 0's
prices = [i.replace('"', '') for i in prices if len(i) > 1]

# Stations dataframe
r = requests.get(url, headers=header)

df = pd.read_html(r.text)[0]




df

,0,1,2,3
0,NaN,Pioneer (158 reviews) 2451 Lakeshore Rd W & ...,Oakville,4h ago BradP542
1,Amenities: Air Pump; Restrooms; C-Store; ATM; ...,NaN,NaN,NaN
2,NaN,Petro-Canada (41 reviews) 845 Burloak Dr & W...,Oakville,1h ago pennymcnally
3,Amenities: Car Wash; Pay At Pump; Open 24/7; C...,NaN,NaN,NaN
4,NaN,Petro-Canada (77 reviews) 2969 Sherwood Heig...,Oakville,38m ago Buddy_2015
5,Amenities: Car Wash; Air Pump; Pay At Pump; Re...,NaN,NaN,NaN
6,NaN,Econo (135 reviews) 2383 Dundas St W near Ol...,Oakville,3h ago willroshev76
7,Amenities: Air Pump; C-Store; Restaurant; Serv...,NaN,NaN,NaN
8,NaN,Petro-Canada (29 reviews) 2300 Royal Windsor...,Oakville,3h ago Lengthy
9,Amenities: Air Pump; Pay At Pump; Restrooms; C...,NaN,NaN,NaN


In [69]:
df1 = df[[1,2,3]].dropna().rename(columns={1:'Station', 2: 'City', 3:'Date'})
city = df1.City.iloc[0]
df1['Prices'] = prices
df1['Address'] = df1.Station.apply(lambda x: x.split(')  ')[1]) 
df1['Address'] = df1.Address.apply(lambda x: x.split(city)[0])
df1['Station'] = df1.Station.apply(lambda x: x.split(' (')[0])
df1['Date'] = df1.Date.apply(lambda x: x.split(' ago')[0])

for date in df1.Date:
    print(now, date)
    if date[-1] == 'h':
        delta = int(date[:-1])
        date = datetime.now() - timedelta(hours=delta)
        print(delta)
    elif date[-1] == 'm':
        delta = int(date[:-1])
        date = datetime.now() - timedelta(minutes=delta)
        print(delta)
    print(date)
df1

2019-05-26 22:52:06.462940 4h
4
2019-05-26 18:52:30.686822
2019-05-26 22:52:06.462940 1h
1
2019-05-26 21:52:30.686822
2019-05-26 22:52:06.462940 38m
38
2019-05-26 22:14:30.686822
2019-05-26 22:52:06.462940 3h
3
2019-05-26 19:52:30.686822
2019-05-26 22:52:06.462940 3h
3
2019-05-26 19:52:30.686822
2019-05-26 22:52:06.462940 4h
4
2019-05-26 18:52:30.686822
2019-05-26 22:52:06.462940 3h
3
2019-05-26 19:52:30.686822
2019-05-26 22:52:06.462940 6h
6
2019-05-26 16:52:30.686822
2019-05-26 22:52:06.462940 35m
35
2019-05-26 22:17:30.687822
2019-05-26 22:52:06.462940 1h
1
2019-05-26 21:52:30.687822


,Station,City,Date,Prices,Address
0,Pioneer,Oakville,4h,121.2,2451 Lakeshore Rd W & Bronte Rd
2,Petro-Canada,Oakville,1h,121.8,845 Burloak Dr & Wyecroft Rd
4,Petro-Canada,Oakville,38m,121.9,2969 Sherwood Heights Dr & South Sheridan Way
6,Econo,Oakville,3h,121.9,2383 Dundas St W near Old Bronte Rd
8,Petro-Canada,Oakville,3h,121.9,2300 Royal Windsor Dr & Ford Dr
10,Esso,Oakville,4h,121.9,541 Maple Grove Dr & Cornwall Rd
12,Pioneer,Oakville,3h,122.0,754 Bronte Rd near QEW
14,7-Eleven,Oakville,6h,123.6,2267 Lakeshore Rd W & East St
16,Petro-Canada,Oakville,35m,123.8,350 Iroquois Shore Rd & Trafalgar Rd
18,Petro-Canada,Oakville,1h,123.8,1007 Upper Middle Rd E & Eighth Line


In [79]:
dfr = df[[1,2,3]].dropna().reset_index(drop=True)
dfr['Prices'] = prices
dfr[1][1]

'Pioneer  (157 reviews)  2451 Lakeshore Rd W & Bronte Rd  Oakville'

In [33]:
import requests

url = 'https://www.gasbuddy.com/GasPrices/Ontario/Oakville'

header = {
  "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
  "X-Requested-With": "XMLHttpRequest"
}

r = requests.get(url, headers=header)

df = pd.read_html(r.text)